<a href="https://colab.research.google.com/github/ChakshuGautam/transformers/blob/main/conversation_terminator_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install tiktoken
!pip install datasets
!pip uninstall -y transformers accelerate
!pip install transformers==4.28.0
!pip install git+https://github.com/huggingface/accelerate
!pip install evaluate
!git clone https://github.com/ChakshuGautam/transformers
%cd transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.28.0-py3-none-any.whl (7.0 MB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-w9qj5ix9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-w9qj5ix9
  Resolved https://github.com/huggingface/accelerate to commit 0226f750257b3bf2cadc4f189f9eef0c764a0467
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... do

In [24]:
from transformers import pipeline
from datasets import list_datasets, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, BertTokenizerFast, TFBertForSequenceClassification, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('google/mobilebert-uncased')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# def tokenize(batch):
#     return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=512)

dataset = load_dataset('Chakshu/conversation_ender')
dataset = dataset["train"]
tokenized_data = tokenizer(dataset["text"], return_tensors="np", padding=True)
tokenized_data = dict(tokenized_data)
labels = np.array(dataset["is_terminator"])

model = TFBertForSequenceClassification.from_pretrained('google/mobilebert-uncased', num_labels=1)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MobileBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


  0%|          | 0/2 [00:00<?, ?it/s]

You are using a model of type mobilebert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at google/mobilebert-uncased were not used when initializing TFBertForSequenceClassification: ['mobilebert', 'predictions___cls', 'seq_relationship___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['be

In [33]:
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=tf.keras.metrics.BinaryAccuracy())

model.fit(tokenized_data, labels, epochs=3, batch_size=32)

Epoch 1/3
8/8 [==============================] - 126s 5s/step - loss: 0.4289 - binary_accuracy: 0.8376
Epoch 2/3
8/8 [==============================] - 27s 3s/step - loss: 0.3591 - binary_accuracy: 0.8632
Epoch 3/3
8/8 [==============================] - 25s 3s/step - loss: 0.2268 - binary_accuracy: 0.9444


In [ ]:
tf.saved_model.save(model, 'save_model')

In [30]:
test_dataset = load_dataset('Chakshu/conversation_ender')
test_dataset = test_dataset["test"]
test_tokenized_data = tokenizer(dataset["text"], return_tensors="np", padding=True)
test_tokenized_data = dict(test_tokenized_data)
labels = np.array(dataset["is_terminator"])

loaded_model = tf.saved_model.load('save_model')

  0%|          | 0/2 [00:00<?, ?it/s]

In [52]:
total_correct = 0

for i in range(len(test_dataset)):
  single_string = test_dataset[i]['text']
  inputs = tokenizer(single_string, return_tensors="np", padding=True)
  predictions = model(inputs.input_ids, inputs.attention_mask)
  probabilities = tf.nn.sigmoid(predictions.logits)

  # Round the probabilities to the nearest integer to get the class prediction
  predicted_class = tf.round(probabilities)

  # Extract the predicted class as a Python integer and print it
  total_correct += int(predicted_class.numpy()) == test_dataset[i]['is_terminator']
  # print(single_string, test_dataset[i]['is_terminator'], " ||| Correct Prediction?", int(predicted_class.numpy()), int(predicted_class.numpy()) == test_dataset[i]['is_terminator'])

print("Total Correct Classifications =", 100*total_correct/len(test_dataset))

Total Correct Classifications =  100.0
